In [1]:
%load_ext autoreload

In [2]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from sandbox import rrtiris

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RigidTransform, RollPitchYaw
import meshcat
from pydrake.all import GenerateSeedingPolytope

# Build Plant


In [4]:
q0 = [0.0, 0.0, 0.0]
q_low  = [-1.7, -2., -1.7]
q_high = [ 1.7,  2.,  1.7]


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/oneDOF_iiwa7_with_box_collision.sdf")
twoDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/twoDOF_iiwa7_with_box_collision.sdf")

In [6]:
box_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(twoDOF_iiwa_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset))



locs = [[0.,0.,0.],[0.,.5,0.],[0.,-.5,0.]]
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", models[0]), RigidTransform(locs[0]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_twoDOF_link_0", models[1]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[1]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[2]))


plant.Finalize()

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
            
#q_low = plant.GetPositionLowerLimits().tolist()
#q_high = plant.GetPositionUpperLimits().tolist()

Ratfk = RationalForwardKinematics(plant)


In [7]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram
visualizer.visualize_collision_constraint(N = 30)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7012/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7013/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6012...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7012/static/
Connected to meshcat-server.


In [8]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    visualizer.showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

visualizer.jupyter_cell()

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [9]:
#filter fused joints self collisions 
digaram = visualizer.diagram
context = visualizer.diagram_context
plant_context = visualizer.plant_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
iiwa_oneDOF_gids = [gid for gid in gids if "iiwa7_oneDOF::" in get_name_of_gid(gid)]
iiwa_twoDOF_gids = [gid for gid in gids if "iiwa7_twoDOF::" in get_name_of_gid(gid)]

oneDOF_fused_col_geom = iiwa_oneDOF_gids[2:]
iiwa_oneDOF_fused_set = GeometrySet(oneDOF_fused_col_geom)
twoDOF_fused_col_geom = iiwa_twoDOF_gids[4:]
iiwa_twoDOF_fused_set = GeometrySet(twoDOF_fused_col_geom)
# print([get_name_of_gid(gid) for gid in oneDOF_fused_col_geom])
# print([get_name_of_gid(gid) for gid in twoDOF_fused_col_geom])
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_oneDOF_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_twoDOF_fused_set))
pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

150 -> 150
137 -> 150


# Setup IRIS Options and Generate Regions

In [10]:
# seed_points_q = np.array([[0.0, 0, 0], # zero config
#                         [0.0, -1.3, -1.3],  # start
#                         [0.9, -1.5, -0.8]     # goal
#                          ])
seed_points_q = np.array([[0.0, 0, 0], # zero config
                        [0.8, 1.3, -0.8],  # START: blue low green up
                        [0.1, 0.9, -1.2],     # GOAL: green low other up
                        [0.2, 1.6, -0.6],
                        [0.5, -1.9, -0.9]
                         ])[:,(0,1,2)]   # passing


seed_points = np.array([Ratfk.ComputeTValue(seed_points_q[idx], np.zeros((3,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)

start = seed_points[1,:]
goal = seed_points[2,:]

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), np.zeros((3,)) ))
    
starting_poly = HPolyhedron.MakeBox(limits_t[0], limits_t[1])
context = diagram.CreateDefaultContext()

In [11]:
regions = []

iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 20
iris_options.configuration_space_margin = 1e-5
iris_options.max_faces_per_collision_pair = 60
iris_options.termination_threshold = -1
iris_options.q_star = np.zeros(3)
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
#deprecated
iris_options.certify_region_with_sos_during_generation = False
iris_options.certify_region_with_sos_after_generation = False

for i, s in enumerate(seed_points):
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
    if False:
        #starting_hpolyhedron = regions[i-1]
        r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
                                              iris_options, starting_hpolyhedron)
    else:
        r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
    regions.append(r)
    print(f'Completed region: {i+1}/{len(seed_points)}')
    print(f"Sample point contained = {np.all(r.A()@s <= r.b())}")
    print(f"Sample point contained componentwise = {r.A()@s <= r.b()}")
    print()

Completed region: 1/5
Sample point contained = True
Sample point contained componentwise = [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]

Completed region: 2/5
Sample point contained = False
Sample point contained componentwise = [ True  True  True  True  True  True  True  True False False False False
  True  True False False  True  True  True  True]

Completed region: 3/5
Sample point contained = False
Sample point contained componentwise = [ True  True  True  True  True  True  True  True  True False False False
 False False False False  True  True False  True  True  True  True  True]

Completed region: 4/5
Sample point contained = False
Sample point contained componentwise = [ True  True  True  True  True  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True

In [12]:
if do_viz:
    visualizer.plot_regions(regions, ellipses=None, region_suffix='original')

In [13]:
regions_default = []
num_perm_dim = 2
num_rot = 2
for i, s in enumerate(seed_points):
    regions_default.append(GenerateSeedingPolytope(s, num_perm_dim, num_rot))
if do_viz:
    visualizer.plot_regions(regions_default, ellipses=None, region_suffix='default')

In [14]:
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.multibody.rational_forward_kinematics import FindEpsilonLower
from pydrake.solvers import mathematicalprogram as mp
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)

scalar_edited_regions = []
vector_edited_regions = []
filtered_collision_pairs = set()

binary_search_options = rational_forward_kinematics.BinarySearchOption()
binary_search_options.epsilon_max = 1
binary_search_options.epsilon_min = -.12
binary_search_options.max_iters = 15
binary_search_options.search_d = True

vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
vector_bisection_search_options.epsilon_max = 1
vector_bisection_search_options.epsilon_min = -.12
vector_bisection_search_options.max_iters = 10
vector_bisection_search_options.search_d = True
vector_bisection_search_options.infeasible_counts_as_iter = False



bilinear_alternation_option = rational_forward_kinematics.BilinearAlternationOption()
bilinear_alternation_option.max_iters = 20
bilinear_alternation_option.lagrangian_backoff_scale = 1e-6
bilinear_alternation_option.polytope_backoff_scale = 1e-6

iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 20
iris_options.configuration_space_margin = 1e-5
iris_options.max_faces_per_collision_pair = 60
iris_options.termination_threshold = -1
iris_options.q_star = np.zeros(3)
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
#deprecated
iris_options.certify_region_with_sos_during_generation = False
iris_options.certify_region_with_sos_after_generation = False

solver_options = mp.SolverOptions()
solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

def iris_scalar_handle(seed, domain):
    seed_q = Ratfk.ComputeQValue(seed, np.zeros((3,))) 
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), seed_q)
    region = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context),
                                              iris_options, domain)
    ellipse = region.MaximumVolumeInscribedEllipsoid()
    C = ellipse.A()
    radii2, R  =  np.linalg.eig(C.T @ C)
    radii = np.sqrt(radii2)
#     binary_search_options.epsilon_min = FindEpsilonLower(region.A(), region.b(),
#                                                          limits_t[0], limits_t[1], 
#                                                          seed)
    binary_search_options.epsilon_min = FindEpsilonLower(region.A(), region.b(),
                                                         limits_t[0], limits_t[1])
    print("[IRIS +  CERT]: Eps min", binary_search_options.epsilon_min)
    print("[IRIS +  CERT]: Region found, vol = ", ellipse.Volume(), ", halfaxis: ", radii)
    b_feasible = cspace_free_region.CspacePolytopeBinarySearch(
                 iris_options.q_star, filtered_collision_pairs, region.A(), region.b(),
                 binary_search_options, solver_options)
    
#     C_final, d_final, P_final, q_final = \
#         cspace_free_region.CspacePolytopeBilinearAlternation(
#             iris_options.q_star, filtered_collision_pairs, region.A(), b_feasible,
#            bilinear_alternation_option, solver_options)
    #C_final = np.vstack([C_final, P_joint_limits.A()])
    #d_final = np.concatenate([d_final, P_joint_limits.b()])
    #editted_regions.append(HPolyhedron(C_final, d_final))
    return region, HPolyhedron(region.A(), b_feasible) 

def iris_vector_handle(seed, domain):
    seed_q = Ratfk.ComputeQValue(seed, np.zeros((3,))) 
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), seed_q)
    region = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context),
                                              iris_options, domain)
    ellipse = region.MaximumVolumeInscribedEllipsoid()
    C = ellipse.A()
    radii2, R  =  np.linalg.eig(C.T @ C)
    radii = np.sqrt(radii2)
#     binary_search_options.epsilon_min = FindEpsilonLower(region.A(), region.b(),
#                                                          limits_t[0], limits_t[1], seed)
    vector_bisection_search_options.epsilon_min = FindEpsilonLower(region.A(), region.b(),
                                                         limits_t[0], limits_t[1])
    print("[IRIS +  CERT]: Eps min", vector_bisection_search_options.epsilon_min)
    print("[IRIS +  CERT]: Region found, vol = ", ellipse.Volume(), ", halfaxis: ", radii)
    b_feasible = cspace_free_region.CspacePolytopeBisectionSearchVector(
                 iris_options.q_star, filtered_collision_pairs, region.A(), region.b(),
                 vector_bisection_search_options, solver_options)
    
#     C_final, d_final, P_final, q_final = \
#         cspace_free_region.CspacePolytopeBilinearAlternation(
#             iris_options.q_star, filtered_collision_pairs, region.A(), b_feasible,
#            bilinear_alternation_option, solver_options)
    #C_final = np.vstack([C_final, P_joint_limits.A()])
    #d_final = np.concatenate([d_final, P_joint_limits.b()])
    #editted_regions.append(HPolyhedron(C_final, d_final))
    return region, HPolyhedron(region.A(), b_feasible) 

In [15]:
i = 0
eps_min = FindEpsilonLower(regions_default[i].A(), regions_default[i].b(), limits_t[0], limits_t[1])
print(eps_min)
r0 = HPolyhedron(regions_default[i].A(), regions_default[i].b()+eps_min*0.9)
s = seed_points[i,:]
if do_viz:
    visualizer.plot_regions([r0], ellipses=None, region_suffix='_small')
# region_init, region = iris_vector_handle(s, region_init)


-0.9999999999970421


In [16]:
print(r0.A())

[[ 1.          0.          0.        ]
 [ 0.          1.          0.        ]
 [ 0.          0.          1.        ]
 [-1.         -0.         -0.        ]
 [-0.         -1.         -0.        ]
 [-0.         -0.         -1.        ]
 [ 0.70710678  0.70710678  0.        ]
 [-0.70710678  0.70710678  0.        ]
 [ 0.          0.          1.        ]
 [-0.70710678 -0.70710678  0.        ]
 [ 0.70710678 -0.70710678  0.        ]
 [ 0.          0.         -1.        ]
 [ 1.          0.          0.        ]
 [ 0.          0.70710678  0.70710678]
 [ 0.         -0.70710678  0.70710678]
 [-1.          0.          0.        ]
 [ 0.         -0.70710678 -0.70710678]
 [ 0.          0.70710678 -0.70710678]
 [ 0.25881905  0.96592583  0.        ]
 [-0.96592583  0.25881905  0.        ]
 [-0.          0.          1.        ]
 [-0.25881905 -0.96592583  0.        ]
 [ 0.96592583 -0.25881905  0.        ]
 [-0.          0.         -1.        ]
 [ 1.         -0.          0.        ]
 [ 0.          0.25881905

In [17]:
binary_search_options.epsilon_min = FindEpsilonLower(r0.A(), r0.b(),
                                                         limits_t[0], limits_t[1])
b_feasible = cspace_free_region.CspacePolytopeBinarySearch(
                 iris_options.q_star, filtered_collision_pairs, r0.A(), r0.b(),
                 binary_search_options, solver_options)
r0_grown_scalar = HPolyhedron(r0.A(), b_feasible) 


[2022-02-01 23:34:08.205] [console] [info] Solver time 2.2550008296966553
[2022-02-01 23:34:10.168] [console] [info] Solver time 0.5598189830780029
[2022-02-01 23:34:15.249] [console] [info] Solver time 2.1895270347595215
[2022-02-01 23:34:15.296] [console] [info] epsilon=0.4500000000001479 is infeasible
[2022-02-01 23:34:21.192] [console] [info] Solver time 3.123392105102539
[2022-02-01 23:34:21.240] [console] [info] epsilon=0.17500000000022187 is infeasible
[2022-02-01 23:34:28.644] [console] [info] bilinear alt on d is successful false
[2022-02-01 23:34:28.656] [console] [info] Solver time 1.3092918395996094
[2022-02-01 23:34:28.724] [console] [info] epsilon=0.03750000000025885 is feasible
[2022-02-01 23:34:28.725] [console] [info] reset eps_min=0.0, eps_max=0.137499999999963
[2022-02-01 23:34:33.337] [console] [info] Solver time 1.8340308666229248
[2022-02-01 23:34:33.387] [console] [info] epsilon=0.0687499999999815 is infeasible
[2022-02-01 23:34:40.949] [console] [info] bilinear 

In [ ]:
vector_bisection_search_options.epsilon_min = FindEpsilonLower(r0.A(), r0.b(),
                                                         limits_t[0], limits_t[1])
b_feasible = cspace_free_region.CspacePolytopeBisectionSearchVector(
                 iris_options.q_star, filtered_collision_pairs, r0.A(), r0.b(),
                 vector_bisection_search_options, solver_options)
r0_grown_vector = HPolyhedron(r0.A(), b_feasible) 

[2022-02-01 23:35:47.656] [console] [info] Solver time 3.346832036972046
[2022-02-01 23:35:53.498] [console] [info] Solver time 1.6442108154296875
[2022-02-01 23:36:00.690] [console] [info] Solver time 3.0940229892730713
[2022-02-01 23:36:00.732] [console] [info] epsilon=0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45
0.45 is infeasible
[2022-02-01 23:36:09.217] [console] [info] Solver time 4.429078102111816
[2022-02-01 23:36:09.260] [console] [info] epsilon=0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175
0.175 is infeasible
[2022-02-01 23:36:20.868] [console] [info] bilinear alt on d is successful true
[2022-02-01 23:36:20.890] [console] [info] Solver time 2.003278970718384
[2022-02-01 23:36:28.798] [console] [info] Solver time 3.8006818294525146
[2022-02-01 23:36:2

[2022-02-01 23:39:12.039] [console] [info] Solver time 3.5449070930480957
[2022-02-01 23:39:12.082] [console] [info] epsilon=0.203229
0.203153
0.203155
0.203154
0.203153
0.203436
0.203154
0.203153
0.203155
0.203153
0.203153
0.203436
0.203229
0.203154
0.209528
0.203154
0.203153
0.230244
 0.69366
0.203153
0.203155
0.203153
0.203154
0.203436
0.203229
0.204017
0.961187
0.203154
0.203154
0.203153 is infeasible
[2022-02-01 23:39:19.168] [console] [info] Solver time 3.0454299449920654
[2022-02-01 23:39:19.212] [console] [info] epsilon=0.146317
0.146236
0.146237
0.146236
0.146236
0.146539
0.146237
0.146236
0.146237
0.146236
0.146236
0.146538
0.146317
0.146236
0.153066
0.146236
0.146236
0.175261
0.671779
0.146236
0.146237
0.146236
0.146237
0.146538
0.146317
0.147161
0.958415
0.146236
0.146237
0.146236 is infeasible
[2022-02-01 23:39:26.168] [console] [info] Solver time 2.9185609817504883
[2022-02-01 23:39:26.211] [console] [info] epsilon=0.117861
0.117777
0.117778
0.117777
0.117777
 0.11809
0.1

[2022-02-01 23:41:55.216] [console] [info] Solver time 2.6693108081817627
[2022-02-01 23:41:55.260] [console] [info] epsilon=0.121737
0.121652
0.121654
0.121652
0.137216
0.122649
0.121653
0.121652
0.121654
0.121652
0.123452
 0.12265
0.121737
0.121653
0.173385
0.121652
0.121741
0.250191
0.957344
0.121652
0.121654
0.121653
0.121653
 0.12265
0.121737
0.122605
 1.80957
0.121652
0.121654
0.121652 is infeasible
[2022-02-01 23:42:02.268] [console] [info] Solver time 2.95934796333313
[2022-02-01 23:42:02.310] [console] [info] epsilon=0.107571
0.107485
0.107487
0.107486
  0.1233
0.108498
0.107486
0.107485
0.107487
0.107485
0.109314
0.108499
0.107571
0.107486
0.160052
0.107486
0.107576
0.238098
0.956656
0.107485
0.107487
0.107486
0.107486
0.108499
0.107571
0.108454
 1.82263
0.107486
0.107487
0.107485 is infeasible
[2022-02-01 23:42:08.794] [console] [info] Solver time 2.563214063644409
[2022-02-01 23:42:08.838] [console] [info] epsilon=0.100488
0.100402
0.100403
0.100402
0.116342
0.101422
0.1004

[2022-02-01 23:44:25.778] [console] [info] epsilon=0.0937383
0.0936505
 0.093652
0.0939371
 0.109711
0.0946788
0.0936516
0.0936502
 0.093652
0.0936515
0.0955077
0.0946796
0.0937383
0.0936513
   0.1891
0.0939372
0.0937428
 0.366041
  1.07367
0.0936503
 0.093652
0.0936509
0.0951612
0.0946796
0.0937383
0.0946338
  2.15443
0.0939372
0.0936522
0.0936506 is infeasible
[2022-02-01 23:44:33.228] [console] [info] Solver time 3.3673369884490967
[2022-02-01 23:44:33.271] [console] [info] epsilon=0.0936277
0.0935398
0.0935413
0.0938265
 0.109602
0.0945682
0.0935409
0.0935396
0.0935413
0.0935409
0.0953973
0.0945691
0.0936277
0.0935407
 0.189001
0.0938265
0.0936322
 0.365963
  1.07368
0.0935396
0.0935413
0.0935402
0.0950508
0.0945691
0.0936277
0.0945232
  2.15457
0.0938265
0.0935415
0.0935399 is infeasible
[2022-02-01 23:44:40.632] [console] [info] Solver time 3.313765048980713
[2022-02-01 23:44:40.675] [console] [info] epsilon=0.0935723
0.0934845
 0.093486
0.0937712
 0.109548
 0.094513
0.0934856
0.

[2022-02-01 23:47:03.245] [console] [info] Solver time 3.2126681804656982
[2022-02-01 23:47:03.289] [console] [info] epsilon=0.0936294
0.0935415
 0.093543
0.0938282
 0.109604
  0.09457
0.0935427
0.0935413
 0.093543
0.0935426
 0.095399
0.0945708
0.0936294
0.0935424
 0.189003
0.0938283
0.0936339
 0.365965
  1.07368
0.0935413
 0.093543
 0.093542
0.0950525
0.0945708
0.0936294
 0.094525
  2.15456
0.0938282
0.0935432
0.0935417 is infeasible
[2022-02-01 23:47:10.528] [console] [info] Solver time 3.185137987136841
[2022-02-01 23:47:10.571] [console] [info] epsilon=0.0935741
0.0934862
0.0934877
0.0937729
 0.109549
0.0945147
0.0934873
 0.093486
0.0934877
0.0934873
0.0953438
0.0945155
0.0935741
0.0934871
 0.188953
0.0937729
0.0935786
 0.365926
  1.07369
 0.093486
0.0934877
0.0934866
0.0949972
0.0945155
0.0935741
0.0944697
  2.15463
0.0937729
0.0934879
0.0934863 is infeasible
[2022-02-01 23:47:18.125] [console] [info] Solver time 3.5099141597747803
[2022-02-01 23:47:18.168] [console] [info] epsilo

In [ ]:
if do_viz:
    visualizer.plot_regions([r0_grown_scalar], ellipses=None, region_suffix='_grown_scalar')
    visualizer.plot_regions([r0_grown_vector], ellipses=None, region_suffix='_grown_vector')

In [15]:
#refine with CPP code:
cpp_editted_vector_regions = []
# for i in range(seed_points.shape[0]):
for i in range(1):
    s = seed_points[i,:]
    region_init = regions[i]
    try:
        region_init, region = iris_vector_handle(s, region_init)
        cpp_editted_vector_regions.append(region)
    except Exception as e:
        print(e)
    print(f"Completed {i+1}/{seed_points.shape[0]}")

[IRIS +  CERT]: Eps min -0.12
[IRIS +  CERT]: Region found, vol =  0.6819787390374958 , halfaxis:  [4.86595118 2.17555524 0.58020292]
CspacePolytopeBisectionSearchVector(): incompatible function arguments. The following argument types are supported:
    1. (self: pydrake.multibody.rational_forward_kinematics.CspaceFreeRegion, q_star: numpy.ndarray[numpy.float64[m, 1]], filtered_collision_pairs: Set[drake::SortedPair<drake::geometry::GeometryId>], C: numpy.ndarray[numpy.float64[m, n], flags.f_contiguous], d_init: numpy.ndarray[numpy.float64[m, 1]], vector_bisection_search_option: pydrake.multibody.rational_forward_kinematics.VectorBisectionSearchOption, solver_options: pydrake.solvers.mathematicalprogram.SolverOptions, q_inner_pts: Optional[numpy.ndarray[numpy.float64[m, n]]] = None, inner_polytope: Optional[Tuple[numpy.ndarray[numpy.float64[m, n]], numpy.ndarray[numpy.float64[m, 1]]]] = None) -> numpy.ndarray[numpy.float64[m, 1]]

Invoked with: <pydrake.multibody.rational_forward_kinem

In [16]:
d_final = np.array([0.939411,
     1.35849,
    0.939411,
    0.939411,
     1.35849,
    0.939411,
   0.0526289,
   0.0260675,
    0.420636,
   0.0328898,
  0.00448928,
   0.0308143,
-0.000379762,
    0.180957,
    0.156639,
    0.129082,
    0.121289,
     0.63852])
d_without_epsilon= np.array([1.13833,
 1.55741,
 1.13833,
 1.13833,
 1.55741,
 1.13833,
 0.25155,
0.224989,
0.619558,
0.231811,
0.203411,
0.229736,
0.198542,
0.379879,
 0.35556,
0.328004,
 0.32021,
0.837441])
eps_min = np.array(d_final-d_without_epsilon)
print(eps_min)

[-0.198919   -0.19892    -0.198919   -0.198919   -0.19892    -0.198919
 -0.1989211  -0.1989215  -0.198922   -0.1989212  -0.19892172 -0.1989217
 -0.19892176 -0.198922   -0.198921   -0.198922   -0.198921   -0.198921  ]


In [16]:
#refine with CPP code:
cpp_editted_scalar_regions = []
for i in range(seed_points.shape[0]):
    s = seed_points[i,:]
    region_init = regions[i]
    region_init, region = iris_scalar_handle(s, region_init)
    cpp_editted_scalar_regions.append(region)
    print(f"Completed {i+1}/{seed_points.shape[0]}")

[2022-01-31 15:40:42.251] [console] [info] Solver time 2.6619720458984375
[2022-01-31 15:40:44.826] [console] [info] Solver time 1.0261502265930176


[IRIS +  CERT]: Eps min -0.2652286679481278
[IRIS +  CERT]: Region found, vol =  0.6819787390374958 , halfaxis:  [4.86595118 2.17555524 0.58020292]


[2022-01-31 15:40:48.683] [console] [info] Solver time 1.619724988937378
[2022-01-31 15:40:48.733] [console] [info] epsilon=-0.1326143339740639 is infeasible
[2022-01-31 15:40:53.321] [console] [info] Solver time 1.117055892944336
[2022-01-31 15:40:53.401] [console] [info] epsilon=-0.19892150096109587 is feasible
[2022-01-31 15:40:53.402] [console] [info] reset eps_min=0.0, eps_max=0.06630716698703196


Completed 1/5
[IRIS +  CERT]: Eps min -0.29420562277149487
[IRIS +  CERT]: Region found, vol =  0.5535987103063078 , halfaxis:  [4.69487125 2.58579857 0.62326848]


[2022-01-31 15:41:02.811] [console] [info] Solver time 3.412208080291748
[2022-01-31 15:41:05.195] [console] [info] Solver time 0.7390580177307129
[2022-01-31 15:41:10.948] [console] [info] Solver time 1.3550069332122803
[2022-01-31 15:41:11.041] [console] [info] epsilon=-0.14710281138574743 is feasible
[2022-01-31 15:41:11.042] [console] [info] reset eps_min=0.0, eps_max=0.14710281138574743
[2022-01-31 15:41:16.878] [console] [info] Solver time 2.747184991836548
[2022-01-31 15:41:16.951] [console] [info] epsilon=0.07355140569287372 is infeasible
[2022-01-31 15:41:24.801] [console] [info] Solver time 3.2756950855255127


...

Completed 4/5


[2022-01-31 15:42:31.487] [console] [info] Solver time 4.768555164337158


[IRIS +  CERT]: Eps min -0.33868521538628993
[IRIS +  CERT]: Region found, vol =  0.43617411313252785 , halfaxis:  [3.98248604 1.65632853 1.45588804]


[2022-01-31 15:42:34.253] [console] [info] Solver time 0.7434499263763428
[2022-01-31 15:42:42.592] [console] [info] Solver time 1.8796329498291016
[2022-01-31 15:42:42.727] [console] [info] epsilon=-0.16934260769314496 is feasible
[2022-01-31 15:42:42.728] [console] [info] reset eps_min=0.0, eps_max=0.16934260769314502
[2022-01-31 15:42:52.439] [console] [info] Solver time 2.5359718799591064
[2022-01-31 15:42:52.575] [console] [info] epsilon=0.08467130384657251 is feasible
[2022-01-31 15:42:52.576] [console] [info] reset eps_min=0.0, eps_max=0.08467130384657251


Completed 5/5


In [40]:
if do_viz:
    visualizer.plot_regions(cpp_editted_regions, ellipses=None, region_suffix='cpp_editted_regions')

In [44]:
#setup refine with python
iris_kwargs = {
    'iris_starting_ellipse_vol': 1e-5,
    'iris_plane_pullback': 1e-4,
    'iris_max_faces': -1
}
iris_generator = CertifiedIrisRegionGenerator(visualizer.diagram, plant, scene_graph, **iris_kwargs)
regions, ellipses = iris_generator.iris_in_rational_space(seed_points)
iris_generator.initalize_certifier(plane_order = 1, strict_pos_tol = 1e-4)

snopt_example=[ 0.35244668 -0.19250022 -0.17979872], growth = 1936025801.4851828
snopt_example=[ 0.23144445 -0.04062289 -0.46224621], growth = 2688883094.3830557
snopt_example=[ 0.2001245  -0.10137361 -0.26895992], growth = 1226658660.3889592
snopt_example=[0.         0.         0.22974536], growth = 527829310.38532484
snopt_example=[-0.18114806  0.0938193   0.        ], growth = 416166830.35149866
snopt_example=[-0.1760171   0.09533373  0.        ], growth = 400705370.23392135
snopt_example=[ 0.2157998  -0.11140991 -0.25108731], growth = 1220265554.750688
0
snopt_example=[ 0.16305709  0.71849721 -0.56308786], growth = 2.899642778747243
snopt_example=[-0.60852286 -1.18066191 -0.12769203], growth = 0.6628366692873271
1
Time:   1.61 	Volume:   0.56 	Center: [ 0.18683057  0.06844461 -0.0630242 ]
snopt_example=[ 0.81716573  0.47417491 -0.38295522], growth = 2389296154.6526375
snopt_example=[ 0.77694698  0.54947603 -0.40266231], growth = 1702365874.8724155
snopt_example=[ 0.44441592  0.7602

{<pydrake.geometry.optimization.HPolyhedron at 0x7f9f7db27530>: <certified_iris_generator.RegionCertificationProblem at 0x7f9f7f87f3d0>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f9f7f379e70>: <certified_iris_generator.RegionCertificationProblem at 0x7f9f7f87f730>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f9f7da92e70>: <certified_iris_generator.RegionCertificationProblem at 0x7f9f7f87fdf0>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f9f7f6fc530>: <certified_iris_generator.RegionCertificationProblem at 0x7f9f7f87fd90>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f9f7dc17570>: <certified_iris_generator.RegionCertificationProblem at 0x7f9f7f87c550>}

In [45]:
if do_viz:
    visualizer.plot_regions(iris_generator.regions, region_suffix = '_python_snopt')

In [ ]:
# refine with python
do_linesearch_cert = True
if do_linesearch_cert:
    iris_generator.certify_and_adjust_regions_by_linesearch(1e-5)

Starting Region 1/5
min_eps = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.56916636 0.77870386 0.56916636 0.56916636 0.77870386 0.56916636
 0.22000147 0.25927221 0.17511844 0.11487268 0.10200084 0.10008813
 0.1746615  0.31397342 0.33108215]
max_eps = [1.13833271 1.55740772 1.13833271 1.13833271 1.55740772 1.13833271
 0.44000293 0.51854441 0.35023687 0.22974536 0.20400167 0.20017626
 0.349323   0.62794684 0.66216431]
Biggest difference: 1.5574077246549023
Smallest difference: 0.20017626488520598
Fixed eps is success

min_eps = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.28458318 0.38935193 0.28458318 0.28458318 0.38935193 0.28458318
 0.11000073 0.1296361  0.08755922 0.05743634 0.05100042 0.05004407
 0.08733075 0.15698671 0.16554108]
max_eps = [0.56916636 0.77870386 0.56916636 0.56916636 0.77870386 0.56916636
 0.22000147 0.25927221 0.17511844 0.11487268 0.10200084 0.10008813
 0.1746615  0.31397342 0.33108215]
Biggest difference: 0.7787038623274511
Smallest 

In [ ]:
if do_viz:
    visualizer.plot_regions(iris_generator.linesearch_regions, region_suffix = '_new_by_linesearch')